### Card Reader 

Method Testing for reading cards from `api.swu-db.com`.

Created by: Jared Wills<br>
Last Updated: Nov 07, 2025

In [1]:
from bs4 import BeautifulSoup
import requests
from json import loads
import csv
import os
import pandas as pd
import numpy as np

# Constants
SETS = [
    ['SOR', 252], 
    ['SHD', 262], 
    ['TWI', 257], 
    ['JTL', 262], 
    ['LOF', 264], 
    ['IBH', 104], 
    ['SEC', 264],
]

In [2]:
'''
download_cards(sets: list)

Description: Accesses swu-db api and returns a dataframe with all of the card
             information from the listed sets.

Param:
    sets: list
        List of lists of SWU set codes and number of cards to download.
        Appropriate Sets: 'SOR', 'SHD', 'TWI', 'JTL', 'LOF', 'IBH', 'SEC'
        See constant `SETS` above. Includes the standard length of each set.
Returns:
    pandas.DataFrame
        Headers: 'Set', 'Number', 'Name', 'Subtitle', 'Type', 'Aspects', 'Traits',
                 'Arenas', 'Cost', 'Power', 'HP', 'FrontText', 'EpicAction',
                 'DoubleSided', 'BackArt', 'BackText', 'Rarity', 'Unique',
                 'Keywords', 'Artist', 'VariantType', 'MarketPrice', 'LowPrice',
                 'FrontArt', 'FoilPrice', 'LowFoilPrice'
'''
def download_cards(sets: list):
    cards = []
    print('Downloading Card Information')
    for set_, set_len in sets:
        for num in range(set_len):
            c = loads(requests.get(f'https://api.swu-db.com/cards/{set_}/{num + 1:03}').text)
            # if 'message' in c.keys():
                # c = card_from_html(set_, num)
            cards.append(c)
            print(f'Completed: {set_}-{num + 1:03}', end='\r')
    cards = pd.DataFrame(cards).set_index(['Set','Number'])
    return cards

In [3]:
'''
download_all_cards_to_csv():

Description: A General Call of the download_cards() function. This includes
             the full list of SWU Sets found in the Global Constant `SETS`.
             Then creates a .csv file containing the data.

Parameters: 
    None

Returns:
    None
'''
def download_all_cards_to_csv():
    cards = download_cards(SETS)
    cards.to_csv('cards.csv', index=True)

In [4]:
'''
card_from_html(set_, num):

Description: For use when call errors arise from the API. Will scrape and parse 
             the .html of the API website to pull the features of any card.

Parameters: 
    set_: str
        The set code of the card to scrape. Valid values are: 'SOR', 'SHD',
        'TWI', 'JTL', 'LOF', 'IBH', 'SEC'
    num: int
        The card number of the card to scrape. 

Returns:
    c: dict
        A dictionary of similar format to the json returned by the API call.
'''
def card_from_html(set_, num):
    c = {}
    html = requests.get(f'https://www.swu-db.com/card/{set_}/{num:03}').text
    soup = BeautifulSoup(html, 'html.parser')
    c['Set'] = set_
    c['Number'] = num
    c_name = soup.find(id='card-title-aspects').get_text().strip().split(' - ')
    c['Name'] = c_name[0]
    if len(c_name) > 1:
        c['Subtitle'] = c_name[1]
    c['Unique'] = True if '⟡' in c['Name'] else False
    c['Name'] = c['Name'].replace('⟡ ', '')
    c_types = [m.get_text() for m in soup.find_all(class_='card-type-span')]
    c['DoubleSided'] = len(c_types) > 1
    if True in ['Ground' in m.title() for m in c_types]:
        c['Arenas'] = 'Ground'
    elif True in ['Space' in m.title() for m in c_types]:
        c['Arenas'] = 'Ground'
    c['Type'] = c_types[0].split('-')[0].strip().title()
    c['Traits'] = [a.get_text() for a in soup.find(class_='card-traits').find_all('a')]
    c['Traits'] = c['Traits'] if len(c['Traits']) > 0 else None
    c['Aspects'] = [a['src'].split('/')[-1].split('.')[0].title() for a in soup.find_all(alt='Aspect Icon')]
    c['FrontArt'] = soup.find(alt='Card Image')['src']
    c['FrontText'] = soup.find(class_='card-text').get_text().strip()
    if c['DoubleSided']:
        c['BackArt'] = soup.find(class_='card-image').find('img')['data-back-image']
        c['BackText'] = soup.find(class_='card-back-text').get_text().strip()
    c['Keywords'] = [m.get_text().strip() for m in soup.find_all(class_='keyword-text')]
    c['EpicAction'] = soup.find(class_='epic-action-text').get_text().strip() if soup.find(class_='epic-action-text') else None
    for item in soup.find_all(class_='card-set-item'):
        if item.find(class_='card-set-title'):
            if '/' in item.find(class_='card-set-title').get_text().strip():
                k1, k2 = item.find(class_='card-set-title').get_text().strip().split('/')
                v1, v2 = item.find(class_='card-set-text').get_text().strip().split('/')
                for k, v in zip([k1, k2], [v1, v2]):
                    c[k.strip()] = v.strip()
            else:
                c[item.find(class_='card-set-title').get_text().strip()] = item.find(class_='card-set-text').get_text().strip()
    rem = []
    for key in c.keys():
        if key not in ['Set','Number','Name','Subtitle','Type','Aspects','Traits','Arenas','Cost','Power','HP','FrontText','EpicAction','DoubleSided','BackArt','BackText','Rarity','Unique','Keywords','Artist','VariantType','MarketPrice','LowPrice','FrontArt','FoilPrice','LowFoilPrice']:
            rem.append(key)
    for key in rem:
        c.pop(key)
    if 'HP' in c.keys() and c['Type'] == 'Upgrade':
        c.pop('HP')
    return c

In [5]:
def download_set_cards(set: list):
    set = [set] if type(set[0]) != type([]) else set
    cards = download_cards(set)
    return cards

In [6]:
def append_set_cards(set: list):
    cards = pd.read_csv('cards.csv').set_index(['Set', 'Number'])
    set = [set] if type(set[0]) != type([]) else set
    cards = pd.concat([cards, download_set_cards(set)])
    return cards

In [7]:
def get_cards_from_csv():
    return pd.read_csv('cards.csv').set_index(['Set','Number'])

In [ ]:
download_all_cards_to_csv()

Completed: SEC-039